# Create a Slice and Add/Remove SSH Keys

This guide outlines the process of granting access to virtual machines (VMs) within your slice to other users. When collaborating with others, it's common to encounter situations where you need to enable them to access the VMs running in your slice. Access to these VMs on FABRIC is managed through SSH keys. This notebook offers methods to incorporate the SSH keys of your collaborators into the VMs within your slice, ensuring they have the necessary access to your VMs.


## Import the FABlib Library


In [ ]:
from fabrictestbed_extensions.fablib.fablib import FablibManager as fablib_manager

fablib = fablib_manager()

fablib.show_config();

## Create the Experiment Slice

The following creates two nodes with basic NICs connected to an isolated local Ethernet.  


In [ ]:
slice_name = 'SSH-Keys'
site = fablib.get_random_site()
print(f"Sites: {site}")

node1_name='Node1'
node2_name='Node2'

network_name='net1'
nic_name = 'nic1'
model_name = 'NIC_Basic'

In [ ]:
from ipaddress import IPv4Network

subnet = IPv4Network("192.168.1.0/24")
available_ips = list(subnet)[1:]

In [ ]:
#Create Slice
slice = fablib.new_slice(name=slice_name)

net1 = slice.add_l2network(name=network_name, subnet=subnet)

# Node1
node1 = slice.add_node(name=node1_name, site=site)
iface1 = node1.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
iface1.set_mode('auto')
net1.add_interface(iface1)

# Node2
node2 = slice.add_node(name=node2_name, site=site)
iface2 = node2.add_component(model='NIC_Basic', name='nic1').get_interfaces()[0]
iface2.set_mode('auto')
net1.add_interface(iface2)


#Submit Slice Request
slice.submit();

### Create and add ssh keys

#### Get the Slice Information

In [ ]:
slice = fablib.get_slice(slice_name)

#### Generate an SSH Key Pair

In [ ]:
import paramiko

username = "rocky"

# Generate a new RSA key pair
key = paramiko.RSAKey.generate(bits=2048)
public_key_string = f"{key.get_name()} {key.get_base64()}"

private_key_path = "id_rsa_test"
with open(private_key_path, "w") as private_key_file:
    key.write_private_key(private_key_file)

#### Add Public Key to a node and verify connectivity


A public SSH key can be incorporated into a node using the following methods:

- Add the public key to a node by supplying the public key string directly:
`node.add_remove_public_key(sliver_public_key=public_key_string)`

- Include a user's own public sliver key by specifying the sliver key name. The Sliver Key Name can be obtained from the 'Sliver' tab under [SSH Keys](https://portal.fabric-testbed.net/experiments#sshKeys).
`node.add_remove_public_key(sliver_key_name=sliver_key_name)`

- Integrate another user's public sliver key by providing their sliver key name and email address. The Sliver Key Name is accessible in the 'Sliver' tab under [SSH Keys](https://portal.fabric-testbed.net/experiments#sshKeys).
`node.add_remove_public_key(sliver_key_name=sliver_key_name, email=email)`

In [ ]:
for node in slice.get_nodes():
    
    # Add Public Key to the Node by passing the public key string
    node.add_remove_public_key(sliver_public_key=public_key_string)
    
    # Add Portal Public Key to the Node by passing the sliver key name
    #sliver_key_name = "REPLACE_WITH_SLIVER_KEY_NAME"
    #node.add_remove_public_key(sliver_key_name=sliver_key_name)
    
    # Add Collaborator's Portal Public Key to the Node by passing the sliver key name and email
    #sliver_key_name = "REPLACE_WITH_SLIVER_KEY_NAME"
    #email = "REPLACE_COLLABORATORS_EMAIL"
    #node.add_remove_public_key(sliver_key_name=sliver_key_name, email=email)
    
    # Validating SSH connection using the key
    # When using the portal keys, this validation would require the respective private key to be saved at
    # a location passed in private_key_path
    try:
        #private_key_path = "REPLACE_WITH_PRIVATE_KEY_PATH_FOR_PORTAL_KEYS"
        stdout, stderr = node.execute('echo Hello, FABRIC from node `hostname -s`', 
                                      private_key_file=private_key_path)
    except Exception as e:
        print("Error:", str(e))

    print()    
    

#### Remove public key from a node

In [ ]:
for node in slice.get_nodes():
    
    # Add Public Key to the Node by passing the public key string
    node.add_remove_public_key(sliver_public_key=public_key_string, remove=True)
    
    # Add Portal Public Key to the Node by passing the sliver key name
    #sliver_key_name = "REPLACE_WITH_SLIVER_KEY_NAME"
    #node.add_remove_public_key(sliver_key_name=sliver_key_name, remove=True)
    
    # Add Collaborator's Portal Public Key to the Node by passing the sliver key name and email
    #sliver_key_name = "REPLACE_WITH_SLIVER_KEY_NAME"
    #email = "REPLACE_COLLABORATORS_EMAIL"
    #node.add_remove_public_key(sliver_key_name=sliver_key_name, email=email, remove=True) 

## Delete the Slice

Please delete your slice when you are done with your experiment.

In [ ]:
slice = fablib.get_slice(slice_name)
slice.delete()